* 求解机器学习的步骤分为学习和推理两个阶段
* 学习阶段是通过训练数据来调整模型的参数，使模型能够更好地拟合数据
* 推理阶段是使用训练好的模型对新数据进行预测

In [ ]:
# coding: utf-8
# 该方法下载 MNIST 数据集，并将其转换为 pickle 文件
# pkl 文件是 Python 的一种序列化文件格式，可以将 Python 对象保存到文件中，方便后续使用
try:
    import urllib.request # 倒入 urllib.request 用于下载文件
except ImportError:
    raise ImportError('You should use Python 3.x')
import os.path # 用于处理文件和目录路径
import gzip # 用于解压缩 gzip 文件
import pickle # 用于序列化和反序列化 Python 对象
import os # 用于处理文件和目录路径
import numpy as np


url_base = 'https://ossci-datasets.s3.amazonaws.com/mnist/'  # MNIST 数据集的下载地址
key_file = {
    'train_img':'train-images-idx3-ubyte.gz', # 训练图像文件名
    'train_label':'train-labels-idx1-ubyte.gz', # 训练标签文件名
    'test_img':'t10k-images-idx3-ubyte.gz', # 测试图像文件名
    'test_label':'t10k-labels-idx1-ubyte.gz' # 测试标签文件名
}

dataset_dir = "/Users/Crimson/PycharmProjects/DeepLearning/pickle"  # 获取当前文件的目录
save_file = dataset_dir + "/mnist.pkl" # 保存 pickle 文件的路径

train_num = 60000 # 训练数据的数量
test_num = 10000 # 测试数据的数量
img_dim = (1, 28, 28) # 图像的维度，1表示灰度图，28x28 表示图像的大小
img_size = 784 # 图像的大小，28x28=784

# 下载文件
def _download(file_name):
    file_path = dataset_dir + "/" + file_name

    if os.path.exists(file_path):
        return

    print("Downloading " + file_name + " ... ")
    urllib.request.urlretrieve(url_base + file_name, file_path)
    print("Done")

# 下载  MNIST 数据集
def download_mnist():
    for v in key_file.values():
       _download(v)

# 读取标签文件，并转换为 NumPy数组
def _load_label(file_name):
    file_path = dataset_dir + "/" + file_name

    print("Converting " + file_name + " to NumPy Array ...")
    with gzip.open(file_path, 'rb') as f: # 以二进制读取 gzip 文件， rb 表示读取二进制文件
            labels = np.frombuffer(f.read(), np.uint8, offset=8) # 从文件中读取数据，并转换为 NumPy 数组， offset=8 表示跳过前 8 个字节，因为前 8 个字节是文件头信息。np.frombuffer 用于将字节数据转换为 NumPy 数组
    print("Done")

    return labels

# 读取图像文件，并转换为 NumPy数组
def _load_img(file_name):
    file_path = dataset_dir + "/" + file_name

    print("Converting " + file_name + " to NumPy Array ...")
    with gzip.open(file_path, 'rb') as f:
            data = np.frombuffer(f.read(), np.uint8, offset=16) # offset=16 表示跳过前 16 个字节，因为前 16 个字节是文件头信息
    data = data.reshape(-1, img_size) # 将数据转换为二维数组，reshape(-1, img_size) 表示将数据转换为行数不确定，列数为 img_size 的二维数组
    print("Done")

    return data

# 将数据转换为 pickle 文件
def _convert_numpy():
    dataset = {}
    dataset['train_img'] =  _load_img(key_file['train_img'])
    dataset['train_label'] = _load_label(key_file['train_label'])
    dataset['test_img'] = _load_img(key_file['test_img'])
    dataset['test_label'] = _load_label(key_file['test_label'])

    return dataset

# 初始化 MNIST 数据集
def init_mnist():
    download_mnist()
    dataset = _convert_numpy()
    print("Creating pickle file ...")
    with open(save_file, 'wb') as f:
        pickle.dump(dataset, f, -1)
    print("Done!")

# 将标签转换为 one-hot 数组, one-hot 数组是指[0,0,1,0,0,0,0,0,0,0]这样的数组, 里面的元素只有一个元素为 1，其他元素为 0，因为 MNIST 数据集有 10 个类别，所以 one-hot 数组的长度为 10。为1的元素表示该样本的类别
def _change_one_hot_label(X):
    T = np.zeros((X.size, 10))
    for idx, row in enumerate(T):
        row[X[idx]] = 1

    return T

# 读入 MNIST 数据集
def load_mnist(normalize=True, flatten=True, one_hot_label=False):
    """读入MNIST数据集

    Parameters
    ----------
    normalize : 将图像的像素值正规化为0.0~1.0
    one_hot_label :
        one_hot_label为True的情况下，标签作为one-hot数组返回
        one-hot数组是指[0,0,1,0,0,0,0,0,0,0]这样的数组
    flatten : 是否将图像展开为一维数组

    Returns
    -------
    (训练图像, 训练标签), (测试图像, 测试标签)
    """
    if not os.path.exists(save_file):
        init_mnist()

    with open(save_file, 'rb') as f:
        dataset = pickle.load(f)

    if normalize:
        for key in ('train_img', 'test_img'):
            dataset[key] = dataset[key].astype(np.float32)
            dataset[key] /= 255.0

    if one_hot_label:
        dataset['train_label'] = _change_one_hot_label(dataset['train_label'])
        dataset['test_label'] = _change_one_hot_label(dataset['test_label'])

    if not flatten:
         for key in ('train_img', 'test_img'):
            dataset[key] = dataset[key].reshape(-1, 1, 28, 28)

    return (dataset['train_img'], dataset['train_label']), (dataset['test_img'], dataset['test_label'])


if __name__ == '__main__':
    init_mnist()


In [ ]:
import sys, os
sys.path.append(os.pardir)  # 为了导入父目录的文件而进行的设定
((x_train, t_train), (x_test, t_test)) = load_mnist(flatten=True, normalize=False) # 读入 MNIST 数据集
print(x_train.shape) # (60000, 784)
print(t_train.shape) # (60000,)
print(x_test.shape)  # (10000, 784)
print(t_test.shape)  # (10000,)

经过前面两个方法的处理，x_train 和 x_test 分别是训练数据和测试数据，t_train 和 t_test 分别是训练标签和测试标签。
* x_train 的形状是 (60000, 784)，表示有 60000 张图片，每张图片有 784 个像素点
* t_train 的形状是 (60000,)，表示有 60000 个标签，每个标签是一个数字，表示图片对应的数字
* x_test 的形状是 (10000, 784)，表示有 10000 张图片，每张图片有 784 个像素点
* t_test 的形状是 (10000,)，表示有 10000 个标签，每个标签是一个数字，表示图片对应的数字

In [ ]:
# 取训练图像的第一张看看
from PIL import Image # 用于处理图像
def img_show(img): # 定义一个函数，用于显示图像，img 是一个一维数组，包含 784 个像素点
    pil_img = Image.fromarray(np.uint8(img)) # 将 NumPy 数组转换为 PIL 图像，np.uint8 表示将数据转换为 8 位无符号整数，即 0-255 之间的整数
    pil_img.show() # 显示图像

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False) # 读入 MNIST 数据集
img = x_train[0] # 取训练图像的第一张
label = t_train[0] # 取训练标签的第一张
print(label) # 打印标签
img = img.reshape(28, 28) # 将一维数组转换为二维数组
img_show(img) # 显示图像

接下来实现推理处理
```
输入的图片是 28x28 的灰度图像，所以输入层有 784 个节点
最终的输出是 0-9 的数字，所以输出层有 10 个节点
定义两个隐藏层
第一个隐藏层有 50 个节点，用于提取图像的特征
第二个隐藏层有 100 个节点，用于进一步提取图像的特征
```

In [11]:
# get_data 函数用于获取 MNIST 数据集的测试数据
def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=True, one_hot_label=False)
    return x_test, t_test

# init_network 函数用于初始化神经网络的权重和偏置
def init_network():
    with open("/Users/Crimson/PycharmProjects/DeepLearning/pickle/sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
    return network

# 定义激活函数 sigmoid 和 恒等函数
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
def identity_function(x):
    return x
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

# predict 函数用于进行前向传播，计算神经网络的输出
def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)
    return y

x, t = get_data() # 获取测试数据
network = init_network() # 初始化神经网络
accuracy_cnt = 0 # 预测正确的数量
for i in range(len(x)): # 遍历所有测试数据
    y = predict(network, x[i]) # 进行前向传播，计算神经网络的输出
    p = np.argmax(y) # 取输出中最大的值的索引, max 和 argmax 的区别是 max 取最大值，argmax 取最大值的索引，这里取索引是因为索引对应的数字就是预测的结果
     # np.argmax(a) 返回数组 a 中最大值的索引
     # np.max(a) 返回数组 a 中的最大值
     # 例如，a = [0.1, 0.5, 0.2]，则 np.argmax(a) 返回 1，np.max(a) 返回 0.5
     # 因为索引 1 对应的值是 0.5，是数组中的最大值
     # 这里的 p 就是神经网络预测的数字
     # t[i] 是真实的标签
    if p == t[i]: # 如果预测值等于真实值，预测正确
        accuracy_cnt += 1 # 预测正确的数量加 1
print("Accuracy:" + str(float(accuracy_cnt) / len(x))) # 打印预测正确的比例

Accuracy:0.9352


In [12]:
# 批处理
# 相当与输入层的一位数组变成二维数组，当用二维数组取前向传播时，一次可以处理多个数据
x, t = get_data() # 获取测试数据
network = init_network() # 初始化神经网络
batch_size = 100 # 批处理的数量
accuracy_cnt = 0 # 预测正确的数量
for i in range(0, len(x), batch_size): # 每次取 batch_size 个数据进行处理
    x_batch = x[i:i+batch_size] # 取出批处理的数据
    y_batch = predict(network, x_batch) # 进行前向传播
    p = np.argmax(y_batch, axis=1) # 取输出中最大的值的索引, axis=1 取每一行的最大值的索引，返回一个一维数组
    accuracy_cnt += np.sum(p == t[i:i+batch_size]) # 计算预测正确的数量 # p == t[i:i+batch_size] 返回一个布尔数组，表示每个预测值是否等于真实值，np.sum() 对布尔数组求和，True 计为 1，False 计为 0
print("Accuracy:" + str(float(accuracy_cnt) / len(x))) #

Accuracy:0.9207
